# RAIL's DNF implementation example

authors: Laura Toribio san Cipriano, Sam Schmidt
last successfully run: Jan 13, 2025

A quick demo of the DNF package in RAIL.


[Need to add more about the algorithm and options here!]

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
#%matplotlib inline 

In [ ]:
import rail
import qp
from rail.core.data import TableHandle
from rail.core.stage import RailStage

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

## Training the informer

In [ ]:
dnf_dict = dict(zmin=0.0, zmax=3.0, nzbins=301, hdf5_groupname='photometry')

We will begin by training the algorithm, to to this we instantiate a rail object with a call to the base class.<br>

In [ ]:
from rail.estimation.algos.dnf import DNFInformer, DNFEstimator
pz_train = DNFInformer.make_stage(name='inform_DNF', model='demo_DNF_model.pkl', **dnf_dict)

Now, let's load our training data, which is stored in hdf5 format.  We'll load it into the Data Store so that the ceci stages are able to access it.

In [ ]:
from rail.utils.path_utils import RAILDIR
trainFile = os.path.join(RAILDIR, 'rail/examples_data/testdata/test_dc2_training_9816.hdf5')
testFile = os.path.join(RAILDIR, 'rail/examples_data/testdata/test_dc2_validation_9816.hdf5')
training_data = DS.read_file("training_data", TableHandle, trainFile)
test_data = DS.read_file("test_data", TableHandle, testFile)

The inform stage for CMNN should not take long to run, it essentially just converts the magnitudes to colors for the training data and stores those as a model dictionary which is stored in a pickle file specfied by the `model` keyword above, in this case "demo_cmnn_model.pkl". This file should appear in the directory after we run the inform stage in the cell below:

In [ ]:
%%time
pz_train.inform(training_data)

We can now set up the main photo-z stage and run our algorithm on the data to produce simple photo-z estimates.  Note that we are loading the trained model that we computed from the inform stage: with the `model=pz_train.get_handle('model')` statement.  We will set `nondetect_replace` to `True` to replace our non-detection magnitudes with their 1-sigma limits and use all colors.<br>

DNF allows three different methods for choosing the distance metric: Euclidean ("ENF" set with `selection_mode` of `0`), Angular ("ANF" set with selection_mode of `1`, this is the default for the stage), and Directional ("DNF" set with `selection_mode` of `2`).

In our first example, let's set the `selection_mode` to "1", which will use the angular distance:

In [ ]:
%%time
pz = DNFEstimator.make_stage(name='DNF_estimate', hdf5_groupname='photometry',
                        model=pz_train.get_handle('model'),
                        selection_mode=1,
                        min_n=15,
                        bad_redshift_val=99.,
                        bad_redshift_err=10.,
                        nondetect_replace=True)
results = pz.estimate(test_data)

DNF calculates its own point estimate, `DNF_Z`, which is stored in the qp Ensemble `ancil` data.  Let's plot that versus the true redshift.  We can also compute the PDF mode for each object and plot that as well:

# NOTE: we should probably say what the DNF_Z is exactly, is it mean of the PDF?

In [ ]:
#zmode = results().ancil['zmode']
zdnf = results().ancil['DNF_Z'].flatten()

In [ ]:
zgrid = np.linspace(0,3,301)
zmode = results().mode(zgrid).flatten()

In [ ]:
zmode

Let's plot the redshift mode against the true redshifts to see how they look:

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(test_data()['photometry']['redshift'],zmode,s=1,c='k',label='DNF mode')
plt.plot([0,3],[0,3],'r--');
plt.xlabel("true redshift")
plt.ylabel("DNF photo-z mode")
plt.ylim(0,3)

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(test_data()['photometry']['redshift'], zdnf, s=1, c='k')
plt.plot([0,3],[0,3], 'r--');
plt.xlabel("true redshift")
plt.ylabel("DNF_Z")
plt.ylim(0,3)

## plotting PDFs

In addition to point estimates, we can also plot a few of the full PDFs produced by DNF using the `plot_native` method of the qp Ensemble that we've created as `results`.  We can specify which PDF to plot with the `key` argument to `plot_native`, let's plot four, the 5th, 1380th, 14481st, and 18871st:

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12,8))
whichgals = [4, 1379, 14480, 18870]
for ax, which in zip(axs.flat, whichgals):
    ax.set_xlim(0,3)
    results().plot_native(key=which, axes=ax)
    ax.set_xlabel("redshift")
    ax.set_ylabel("p(z)")

We see quite a bit of structure in the DNF PDFs, with lots of discrete peaks. 


## NOTE: we should probably add more about DNF and why the PDFs look this way with all of the little bumps!!!!!


# Other distance metrics

Besides "DNF" there are options for "ENF" and "ANF" (I don't know what the differences are, looks to be the metric used, and "direction", "euclidean", and "angular".  We would have to read the paper for details), try these out.

Let's run our estimator using `selection_mode=0` for the Euclidean distance, and compare both the mode results and PDF results:

In [ ]:
%%time
pz2 = DNFEstimator.make_stage(name='DNF_estimate2', hdf5_groupname='photometry',
                        model=pz_train.get_handle('model'),
                        selection_mode=0,
                        min_n=15,
                        bad_redshift_val=99.,
                        bad_redshift_err=10.,
                        nondetect_replace=True)
results2 = pz2.estimate(test_data)

In [ ]:
zdnf2 = results2().ancil['DNF_Z'].flatten()

In [ ]:
zgrid = np.linspace(0,3,301)
zmode2 = results2().mode(zgrid).flatten()

First, we can plot the DNF and mode plots on their own, they look very similar, but not exactly the same, as the "angular" distance estimates:

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(test_data()['photometry']['redshift'],zmode2,s=1,c='k',label='DNF mode')
plt.plot([0,3],[0,3],'r--');
plt.xlabel("true redshift")
plt.ylabel("DNF photo-z mode")
plt.ylim(0,3)

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(test_data()['photometry']['redshift'], zdnf2, s=1, c='k')
plt.plot([0,3],[0,3], 'r--');
plt.xlabel("true redshift")
plt.ylabel("DNF_Z")
plt.ylim(0,3)

Let's directly compare the "angular" and "Euclidean" distance estimates on the same axes:

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(test_data()['photometry']['redshift'], zdnf, s=2, c='k', label="angular")
plt.scatter(test_data()['photometry']['redshift'], zdnf2, s=1, c='r', label="Euclidean")
plt.legend(loc='upper left', fontsize=10)
plt.plot([0,3],[0,3], 'm--');
plt.xlabel("true redshift")
plt.ylabel("DNF_Z")
plt.ylim(0,3)

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(test_data()['photometry']['redshift'], zmode, s=2, c='k')
plt.scatter(test_data()['photometry']['redshift'], zmode2, s=1, c='r')
plt.plot([0,3],[0,3], 'm--');
plt.xlabel("true redshift")
plt.ylabel("DNF_Z")
plt.ylim(0,3)

Finally, let's directly compare the same PDFs that we plotted above

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12,8))
whichgals = [4, 1379, 14480, 18870]
for ax, which in zip(axs.flat, whichgals):
    ax.set_xlim(0,3)
    results().plot_native(key=which, axes=ax, label="angular")
    results2().plot_native(key=which, axes=ax, label="Euclidean")
    ax.set_xlabel("redshift")
    ax.set_ylabel("p(z)")
ax.legend(loc='upper left', fontsize=12)

Similar to the mode and DNF point estimates, we see that the PDFs look similar, but not exactly the same.